# Lineage tracing data analysis (Matt)

This chapter serves as an introduction to the basic technologies, tools, and analysis pipelines for lineage tracing.

In the introduction, we will provide the background and motivation into lineage tracing apporaches.

# Lineage tracing technologies (Matt)
Here we will discuss the existing approaches for performing lineage tracing, as well provide an overview of the major considerations for each approach. We will emphasize "evolving" lineage tracers, such as those with CRISPR/Cas9 technologies.

Emphasize the inputs / outputs of the lineage tracing.

# Overview of analysis pipelines (Matt)

Before jumping into the code, we'll give readers an overview of what to expect in the analysis pipeline.

## Environment setup.
We'll focus on using Cassiopeia as the main analysis engine and third parties around tree analysis (Moscot)

# Case Study: Tracing tumor development in a mouse model of lung cancer (Matt)
We'll provide a brief background into the dataset. We'll use the study presented in [Yang et al, Cell (2022)](https://www.cell.com/cell/pdf/S0092-8674(22)00462-7.pdf)

## Preprocessing raw data
We'll discuss the major preprocessing steps needed to go from raw data to character matrices, that will be used for tree reconstruction.

## Reconstructing lineages
We'll discuss the algorithms available for tree reconstruction and discuss the pros/cons of each algorithm. We'll also detail best practcies for tree reconstruction.

## Interpreting tree structure
We'll demonstrate useful approaches for quantifying interesting properties on trees (e.g., expansion, fitness)

## Learning from the tree
We'll discuss how one can integrate transcriptomic data to derive insights into evolutionary patterns. Topics to discuss are:

### Plasticity



### Coupling analysis

# Additional tools for integrating with transcriptomic data (Zoe)

# Conclusions, more resources, future directions (Matt / Zoe)

## Benchmarking / Simulations

## Recap on Dos & Don'ts